In [1]:
import numpy as np
import cv2
import random
import tensorflow as tf
import shutil
from keras_facenet import FaceNet
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras import Sequential

In [243]:
import tensorflow as tf


#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense, GlobalAveragePooling2D, ZeroPadding2D, Reshape
from tensorflow.keras import Model

In [242]:
import tensorflow as tf


In [94]:
from numpy.random import seed
seed(1)

tf.random.set_seed(1)

### From Scratch

In [30]:
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [31]:
#stem of the model
input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

In [32]:
# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
     x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
output = Dense (units = 1000, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_259 (Bat (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_239 (ReLU)             (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_117 (Depthw (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_260 (Bat (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_240 (ReLU)             (None, 112, 112, 32)    

In [100]:
len(model.layers)

84

In [128]:
# downloaded

mobile = tf.keras.applications.mobilenet.MobileNet()
print(len(mobile.layers))
mobile.summary()

93
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112,

In [129]:
x=mobile.layers[-6].output


output=Dense(units=4, activation='softmax')(x)

model=Model(inputs= mobile.input,outputs=output)
model.summary()


Model: "functional_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)    

In [130]:
for layer in model.layers[:-5]:      #23 layers--initially; 
    layer.trainable=False
model.summary()

Model: "functional_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)    

In [131]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, restore_best_weights=True),
]

In [132]:
mobilenet =model.fit(x=train_set, validation_data=valid_set, steps_per_epoch = 18, validation_steps=18,  epochs=100, verbose=2)  # 10.47 - -

Epoch 1/100
18/18 - 9s - loss: 0.4130 - accuracy: 0.8667 - val_loss: 1.0768 - val_accuracy: 0.7611
Epoch 2/100
18/18 - 8s - loss: 0.2143 - accuracy: 0.9486 - val_loss: 0.0832 - val_accuracy: 0.9667
Epoch 3/100
18/18 - 8s - loss: 0.1483 - accuracy: 0.9611 - val_loss: 0.7925 - val_accuracy: 0.8500
Epoch 4/100
18/18 - 8s - loss: 0.0594 - accuracy: 0.9833 - val_loss: 0.1351 - val_accuracy: 0.9667
Epoch 5/100
18/18 - 8s - loss: 0.0858 - accuracy: 0.9889 - val_loss: 0.5072 - val_accuracy: 0.8722
Epoch 6/100
18/18 - 8s - loss: 0.0920 - accuracy: 0.9714 - val_loss: 1.4102 - val_accuracy: 0.8111
Epoch 7/100
18/18 - 8s - loss: 0.1818 - accuracy: 0.9500 - val_loss: 0.5770 - val_accuracy: 0.8500
Epoch 8/100
18/18 - 9s - loss: 0.1429 - accuracy: 0.9778 - val_loss: 1.7853 - val_accuracy: 0.6611
Epoch 9/100
18/18 - 9s - loss: 0.1303 - accuracy: 0.9611 - val_loss: 0.6021 - val_accuracy: 0.8222
Epoch 10/100
18/18 - 9s - loss: 0.1606 - accuracy: 0.9611 - val_loss: 0.6105 - val_accuracy: 0.8333
Epoch 11/

KeyboardInterrupt: 

In [37]:
project_path='C:\\Users\\hp\\Documents\\Data Science\\SEM 3\\Major Project\\Datasets\\DD Data\\data-flair\\'  
files =  os.listdir(project_path)

train_path=os.path.join(project_path,os.listdir(project_path)[1])

valid_path = os.path.join(project_path,os.listdir(project_path)[2])

test_path=os.path.join(project_path,os.listdir(project_path)[0])

In [90]:
train_set=ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = train_path, target_size=(224,224), batch_size=10)

valid_set =ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = valid_path, target_size=(224,224), batch_size=10)

test_set =ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = test_path, target_size=(224,224), batch_size=10)

Found 2865 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
Found 200 images belonging to 4 classes.


### The entire MobileNet Model implementation

In [265]:
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x
#stem of the model
input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
     x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)


# x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
x = GlobalAveragePooling2D()(x)
x = Reshape((1,1,1024))(x)

x = Conv2D(filters = 1024, kernel_size = 1, strides = 1)(x)
x = GlobalAveragePooling2D()(x)


output = Dense (units = 4, activation = 'softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()


Model: "functional_131"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_151 (InputLayer)       [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_1581 (Conv2D)         (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_2720 (Ba (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_2858 (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_1370 (Depth (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_2721 (Ba (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_2859 (ReLU)            (None, 112, 112, 32)   

In [266]:
for layer in model.layers[:-15]:      #23 layers--initially; 
    layer.trainable=False
model.summary()

Model: "functional_131"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_151 (InputLayer)       [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_1581 (Conv2D)         (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_2720 (Ba (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_2858 (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_1370 (Depth (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_2721 (Ba (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_2859 (ReLU)            (None, 112, 112, 32)   

In [267]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1, restore_best_weights=True),
]

In [111]:
# accuracy is very low and almost const when more Dense layers are there b/w Conv and output
# accuracy is very low and almost const when only 1  Dense layers is there b/w Conv and output


In [268]:
mobilenet =model.fit(x=train_set, validation_data=valid_set, steps_per_epoch = 10, validation_steps=10,  epochs=100, verbose=2)  # 10.47 - -

Epoch 1/100
10/10 - 9s - loss: 1.3968 - accuracy: 0.1600 - val_loss: 1.3924 - val_accuracy: 0.2200
Epoch 2/100
10/10 - 6s - loss: 1.3959 - accuracy: 0.2500 - val_loss: 1.3920 - val_accuracy: 0.2400
Epoch 3/100
10/10 - 5s - loss: 1.3871 - accuracy: 0.2000 - val_loss: 1.3886 - val_accuracy: 0.2400
Epoch 4/100
10/10 - 5s - loss: 1.3875 - accuracy: 0.3000 - val_loss: 1.3862 - val_accuracy: 0.3100
Epoch 5/100
10/10 - 5s - loss: 1.3773 - accuracy: 0.3000 - val_loss: 1.3932 - val_accuracy: 0.2700
Epoch 6/100
10/10 - 5s - loss: 1.3902 - accuracy: 0.3000 - val_loss: 1.3946 - val_accuracy: 0.2700
Epoch 7/100
10/10 - 5s - loss: 1.3839 - accuracy: 0.2600 - val_loss: 1.4018 - val_accuracy: 0.2500
Epoch 8/100
10/10 - 5s - loss: 1.3890 - accuracy: 0.2200 - val_loss: 1.3941 - val_accuracy: 0.2600
Epoch 9/100
10/10 - 5s - loss: 1.3785 - accuracy: 0.2600 - val_loss: 1.4077 - val_accuracy: 0.2200
Epoch 10/100
10/10 - 5s - loss: 1.3942 - accuracy: 0.2400 - val_loss: 1.3921 - val_accuracy: 0.2600
Epoch 11/

10/10 - 6s - loss: 1.3853 - accuracy: 0.3200 - val_loss: 1.3861 - val_accuracy: 0.2400
Epoch 84/100
10/10 - 5s - loss: 1.3874 - accuracy: 0.2000 - val_loss: 1.3859 - val_accuracy: 0.2800
Epoch 85/100
10/10 - 6s - loss: 1.3869 - accuracy: 0.1900 - val_loss: 1.3855 - val_accuracy: 0.2500
Epoch 86/100
10/10 - 6s - loss: 1.3871 - accuracy: 0.2600 - val_loss: 1.3880 - val_accuracy: 0.1800
Epoch 87/100
10/10 - 8s - loss: 1.3875 - accuracy: 0.2100 - val_loss: 1.3883 - val_accuracy: 0.1700
Epoch 88/100
10/10 - 9s - loss: 1.3857 - accuracy: 0.2600 - val_loss: 1.3862 - val_accuracy: 0.2700
Epoch 89/100
10/10 - 9s - loss: 1.3847 - accuracy: 0.2700 - val_loss: 1.3882 - val_accuracy: 0.1900
Epoch 90/100
10/10 - 9s - loss: 1.3886 - accuracy: 0.2800 - val_loss: 1.3875 - val_accuracy: 0.2600
Epoch 91/100
10/10 - 9s - loss: 1.3849 - accuracy: 0.2200 - val_loss: 1.3865 - val_accuracy: 0.2700
Epoch 92/100
10/10 - 9s - loss: 1.3875 - accuracy: 0.1500 - val_loss: 1.3898 - val_accuracy: 0.2300
Epoch 93/100


### VGG16

In [33]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    return model

In [34]:
model=VGG16()
model.summary()


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_157 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_159 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_160 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_161 (Conv2D)          (None, 56, 56, 256)      

In [35]:
vgg16_model= tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     